In [1]:
import pandas as pd

# TODO: update path
path = '../content/csv/nwac/east_slopes_north'

# TODO: update with the names of your new files
# [ 2022_file.csv, 2023_file.csv, output_file.csv ]
#   If you want to automate this go right ahead
areas = [
    ['Mazama_2170_2022.csv', 'Mazama_2170_2023.csv', 'Mazama_2170.csv'], 
]

"""
areas = [
    ['NWAC_CrystalSummit_6830_2022.csv', 'NWAC_CrystalSummit_6830_2023.csv', 'NWAC_CrystalSummit_6830'], 
    ['NWAC_MtStHelens-Coldwater_3260_2022.csv', 'NWAC_MtStHelens-Coldwater_3260_2023.csv', 'NWAC_MtStHelens-Coldwater_3260'], 
    ['NWAC_WhitePass-PigtailPeak_5970_2022.csv', 'NWAC_WhitePass-PigtailPeak_5970_2023.csv', 'NWAC_WhitePass-PigtailPeak_5970']
]
"""

nwac_datasets = []

for area in areas:
    df_22 = pd.read_csv(path + area[0], comment='#')
    df_23 = pd.read_csv(path + area[1], comment='#')

    df_22 = df_22.rename(columns={'Date/Time (PST)': 'DateTime'})
    df_23 = df_23.rename(columns={'Date/Time (PST)': 'DateTime'})

    df_22.set_index('DateTime')
    df_23.set_index('DateTime')

    og_df = pd.concat((df_23, df_22))
    rev = og_df.iloc[::-1]

    rev = rev.set_index('DateTime')

    rev.to_csv(path + 'new_datasets/' + area[2] + '.csv', sep=',')

    nwac_datasets.append(rev)



nwac_datasets[0].head()


,Battery Voltage (v),Relative Humidity (%),Temperature (deg F),Wind Direction (deg.),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Speed Minimum (mph)
DateTime,,,,,,,
2022-01-01 00:00:00,13.22,88.50,5.062,283.9,20.71,24.71,17.65
2022-01-01 01:00:00,13.30,88.20,5.779,275.1,18.76,24.00,13.41
2022-01-01 02:00:00,13.24,88.80,6.351,272.1,18.82,23.30,11.30
2022-01-01 03:00:00,13.30,77.99,9.060,273.4,20.01,41.65,15.53
2022-01-01 04:00:00,13.30,58.97,9.660,270.6,20.01,24.71,14.83


In [9]:


def get_mean_nwac():
    """Load all the nwac datasets and concatenate them with their mean"""
     # Load the 4 snotel datasets with extra columns created as above
    # dataframes = [get_snotel_df(file=f) for f in files]
    # formatted = [take_snotel_df_and_calculate_columns(df) for df in dataframes]

    # Combine them and get the average of all numerical values
    # concat = pd.concat(formatted).groupby(level=0)

    # return concat
    pass

for og_df in nwac_datasets:
    for col in ['Battery Voltage  (v) ', 'Battery Voltage  (v) ', 'Wind Speed Minimum  (mph) ', 'Relative Humidity (%) ', 'Precipitation  (") ', 'Relative Humidity  (%) ', 'Temperature  (deg F) ', 'Relative Humidity  (%) ', 'Precipitation  (") ', 'Wind Direction  (deg.) ', 'Wind Speed Average  (mph) ']:      
        try:
            og_df.drop(col, axis=1, inplace=True)       
        except Exception as err:
            pass

for col in nwac_datasets[2].columns:
    print('|' + col + '|')

nwac_datasets[2].head()

# Max Windspeed 24hr

from collections import deque

hourly_wind_readings = deque()
wind_dfs = []

for og_df in nwac_datasets:

    new_df = pd.DataFrame()
    new_df['Date'] = None
    new_df['Max Windspeed 24hr'] = None

    new_df = new_df.set_index('Date')

    for idx, row in og_df.iterrows():
        date = idx.split(' ')[0]
        hour = idx.split(' ')[1]
        if hour == '00:00:00' and len(hourly_wind_readings) >= 24:
            m = -1
            for w in hourly_wind_readings:
                if w > m:
                    m = w
            new_df.loc[date, 'Max Windspeed 24hr'] = m
        if len(hourly_wind_readings) >= 24:
            hourly_wind_readings.pop()
        hourly_wind_readings.appendleft(row['Wind Speed Maximum  (mph) '])

    wind_dfs.append(new_df)


final_wind = pd.concat(wind_dfs).groupby(level=0).mean()
final_wind['Was High Winds 24hr'] = final_wind.apply(lambda row : 1 if row['Max Windspeed 24hr'] >= 25 else 0, axis=1)
final_wind.to_csv('../content/csv/windspeeds.csv', sep=',')

|Wind Speed Maximum  (mph) |
